Research Question: Can student-written RateMyProfessors comments be used to predict numerical professor ratings?

The dataset used in this project will be a publicly available data set from Huggingface, you can access the dataset from this link: https://huggingface.co/datasets/ZephyrUtopia/ratemyprofessors_reviews
If you see GPT anywhere please understand that I took some help from GPT to write that piece of code

Load the dataset from Hugging Face

In [ ]:
from datasets import load_dataset
#Load the dataset from the site of Hugging Face
dataset = load_dataset("ZephyrUtopia/ratemyprofessors_reviews")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['name', 'text', 'rating', 'difficulty', 'date'],
        num_rows: 336239
    })
    test: Dataset({
        features: ['name', 'text', 'rating', 'difficulty', 'date'],
        num_rows: 144103
    })
})


Convert the dataset to pandas array

In [18]:
import pandas as pd
#GPT =, making a dataframe df which has all the data from the dataset of Hugging face
#as pandas array
df = pd.concat(
    [dataset[split].to_pandas() for split in dataset.keys()],
    ignore_index=True
)
print(df.columns)
print(df.shape)

Index(['name', 'text', 'rating', 'difficulty', 'date'], dtype='object')
(480342, 5)


Some basic printing functions to see some of the observations

In [ ]:
#Don't get confused with the output, in some of the cases when pandas cannot squeeze data, it divides
#the data in the way as shows in the output
print(df)
print(df["text"].head())
print(df["rating"].value_counts())

                        name  \
0               Leila Hawana   
1              Gloria Magana   
2          Teresa Canganelli   
3                Kumar Ankit   
4               Irina Garces   
...                      ...   
480337        Crystal Koenig   
480338        Karen Caldwell   
480339        Andrew Hudnall   
480340  David-James Gonzalez   
480341          Tyler Vouros   

                                                     text rating difficulty  \
0       Although the homework is not too much, they ar...    4.0        5.0   
1       Where can I start she's a very tough grader. H...    1.0        5.0   
2       She is a wonderful professor! Very interactive...    5.0        4.0   
3       Study solutions had incorrect answers taken fr...    1.0        4.0   
4       Super nice and helpful. Was always there in of...    5.0        4.0   
...                                                   ...    ...        ...   
480337  Amazing! Dr. Koenig gives the best lectures of...    5